In [1]:
import sys
import os
import subprocess
from string import Template
import shutil

NUM_WORKERS = 2
os.environ['PDSH_SSH_ARGS_APPEND'] = ''
os.environ['NCCL_IB_DISABLE'] = '1'
os.environ['NCCL_IBEXT_DISABLE'] = '1'
os.environ['NCCL_SOCKET_IFNAME'] = 'eth0'
os.environ['NCCL_DEBUG'] = 'INFO'
      
MASTER_IP = os.environ["CDSW_IP_ADDRESS"]
MASTER_PORT = "1234"
trainlogs = "trainlogs"
train_script="ft-LoRA-merge.py"
#train_script="peft_lora_seq2seq.py"
myhostfile="hostfile.txt"
deepspeed_cfg="dsconfig/zero2auto.jso"
#deepspeed_cfg="ds_zero3_cpu.yaml"
worker_gpu=1  
sshd_port=2222

deepspeed_cmd = "deepspeed --hostfile $myhostfile \
--launcher pdsh \
--num_gpus $worker_gpu --num_nodes $NUM_WORKERS \
--master_addr $MASTER_IP \
--ssh_port $sshd_port $train_script \
--deepspeed $deepspeed_cfg"

cml_cmd=Template(deepspeed_cmd).substitute(myhostfile=myhostfile, train_script=train_script,
worker_gpu=worker_gpu, NUM_WORKERS=NUM_WORKERS, sshd_port=sshd_port, MASTER_IP=MASTER_IP, MASTER_PORT=MASTER_PORT,
deepspeed_cfg=deepspeed_cfg)

In [ ]:
!deepspeed --hostfile {myhostfile} \
--launcher pdsh \
--num_gpus {worker_gpu} --num_nodes {NUM_WORKERS} \
--master_addr {MASTER_IP} \
--ssh_port {sshd_port} {train_script} \
--per_device_train_batch_size 1 \
--deepspeed {deepspeed_cfg} \
--source_lang en --target_lang ro

In [ ]:
def remove_dir(dir_path):
    try:
        shutil.rmtree(dir_path)
        print(f"Folder '{dir_path}' has been deleted.")
    except Exception as e:
        # Ignore errors, you can print a message if needed
        print(f"Folder '{dir_path}' has been deleted.")
        
base_model = "bloom-1b1"
base_model_name = "bloom-1b1"
merged_model = "merged_bloom-1b1"
training_output = "training_bloom-1b1"
remove_dir(training_output) 
remove_dir(merged_model)
remove_dir(trainlogs)

In [ ]:
main_cmd = subprocess.Popen([f'bash -c "{cml_cmd}" '], shell=True)
main_cmd.communicate()

In [ ]:
# Please restart the iPython kernel manually.

In [ ]:
!deepspeed --hostfile {myhostfile} \
--launcher pdsh \
--num_gpus {worker_gpu} --num_nodes {NUM_WORKERS} \
--master_addr {MASTER_IP} \
--ssh_port {sshd_port} run_translation.py \
--model_name_or_path t5-large --per_device_train_batch_size 1 \
--deepspeed {deepspeed_cfg} \
--output_dir output_dir --overwrite_output_dir --fp16 \
--do_train --max_train_samples 500 --num_train_epochs 1 \
--dataset_name wmt16 --dataset_config "ro-en" \
--source_lang en --target_lang ro